In [7]:
import pandas
import numpy as np
import tensorflow.keras as keras
from keras.models import Model
from keras.layers import Dense, Input
from IPython.display import display
import sympy as sp
sp.init_printing(use_latex = True)
import math

import matplotlib.pyplot as plt
%matplotlib inline


#Decoding Board From Data
board_size = 9 * 9
input_ape = 81
board_length = int(math.sqrt(board_size))

In [8]:
#defining input shape (board state)
#input_size = X.shape[1]

input_shape = (82, )

# Create the model here. It has to be the same size as the weights that are loaded later in main(). This is a smaller network I made to try to reduce training times
model = keras.models.Sequential()
model.add(keras.layers.Dense(82, input_shape = input_shape))
model.add(keras.layers.Reshape((82, 1)))
model.add(keras.layers.ZeroPadding1D(padding = 9))
model.add(keras.layers.Reshape((10, 10, 1)))
model.add(keras.layers.Conv2D(100, kernel_size = (5, 5), activation = 'relu', padding = 'same', input_shape = input_shape))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Conv2D(100, kernel_size = (5, 5), activation = 'relu', padding = 'same', input_shape = input_shape))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(100, kernel_size = (5, 5), activation = 'relu', padding = 'same', input_shape = input_shape))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(82, activation = 'softmax'))


model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer = keras.optimizers.Adam(), metrics = [keras.metrics.CategoricalAccuracy()])

model.load_weights('mini_weights.h5')
#0 if the game ends
#1 - 81
#row major order from top left, bottom right is 81

In [9]:
import numpy as np
from IPython.display import clear_output # Used to clear previous prints for the ascii board
# Some Constants
WHITE = 1
BLACK = -1
EMPTY = 0
# Use to change board size
WIDTH = 9
BOARD_SIZE = WIDTH*WIDTH

def index_to_coordinate(index): #Returns row, column
    return int(index / WIDTH), index % WIDTH

def coordinate_to_index(x, y): #/Returns index
    return y*WIDTH+x


In [10]:
def encode(board): # Converts a board position into a string, to reduce the memory needed to store past position states
    color = board[-1]
    space = 0
    out = str(color)
    for i in range(WIDTH*WIDTH): # Loop through each tile on the board and encode it
        if board[i] != 0: # Non-Empty Tile
            if board[i] == BLACK: # Black piece
                out += "b"
            if board[i] == WHITE: # White piece
                out += "w"
            if space > 0: # Show that the previous streak of empty tiles has ended.
                out += "." + str(space) + "."
                space = 0
        else: # Space tells how many empty squares there were in a row before a piece was found. An empty board would be encoded as "" (best case), a full board as wbwbwbwb. . . (worst case)
            space += 1
    return out

def decode(notation): # Decodes a string created by the encode function and converts it to an array of length 81.
    index = 1
    strIndex = 0
    skip = 0
    board = np.zeros((WIDTH*WIDTH+1))
    board[-1] = notation[0] # Color of player to move
    board[:] = -1
    while index < WIDTH*WIDTH and strIndex < notation.size():
        if notation[strIndex] == 'b' or notation[strIndex] == 'w':
            if notation[strIndex] == 'b': # Black Piece on this tile
                board[index] = BLACK
            else:                         # White piece on this tile
                board[index] = WHITE
            index += 1
        elif notation[strIndex] == '.':   # Empty tiles
            skip = 0
            strIndex += 1
            while notation[strIndex] != '.': # Streaks of empty tiles are coded as numbers followed by a period. This reads how many empty tiles there were, stopping once it sees a period.
                skip *= 10
                skip += notation[strIndex] - '0'
                strIndex += 1
            index += skip
    return board

In [11]:
def Flood(board, index, color): # Performs a flood fill to check if tiles are completely surrounded. If they are, then it returns a 1 and a list of the pieces to be captured.
    closed = [] # Closed List
    open = [index] # Open List
     # Board Size
    x = index % WIDTH # X coordinate
    y = int(index / WIDTH) # Y coordinate
    if board[index] == color: # THIS FUNCTION ISN'T WORKING PROPERLY. Something went wrong when I tried to make move 0 represent a pass and 1-81 be indexes. I suspect the problem's in the Move() function
        return -1, []
    while len(open) > 0:
        x = open[-1] % WIDTH
        Y = int(open[-1] / WIDTH)
        closed.append(open.pop())
        if x > 0 and board[x-1+y*WIDTH] == color:
            if not x-1+y*WIDTH in closed and not x-1+y*WIDTH in open:
                open.append(x-1+y*WIDTH)
        elif x > 0 and board[x-1+y*WIDTH] == EMPTY:
            return -1, []
        if x < WIDTH-1 and board[x+1+y*WIDTH] == color:
            if not x+1+y*WIDTH in closed and not x+1+y*WIDTH in open:
                open.append(x+1+y*WIDTH)
        elif x < WIDTH-1 and board[x+1+y*WIDTH] == EMPTY:
            return -1, []
        if y > 0 and board[x+(y-1)*WIDTH] == color:
            if not x+(y-1)*WIDTH in closed and not x+(y-1)*WIDTH in open:
                open.append(x+(y-1)*WIDTH)
        elif y > 0 and board[x+(y-1)*WIDTH] == EMPTY:
            return -1, []
        if y < WIDTH-1 and board[x+(y+1)*WIDTH] == color:
            if not x+(y+1)*WIDTH in closed and not x+(y+1)*WIDTH in open:
                open.append(x+(y+1)*WIDTH)
        elif y < WIDTH-1 and board[x+(y+1)*WIDTH] == EMPTY:
            return -1, []
    return 1, closed

def Move(bd, index): # Make a move. Return 1, board if successful, -1, parameter board if unsuccessful (ie suicide move)
    # Make a move at the index. color 1 = white, color -1 = black
    color = bd[-1]
    ENEMY = color * -1 # Opposite color of the player's
    board = np.array(bd) # This is done to make a shallow copy of the parameter
    board[-1] *= -1
    board[index] = color # This is where the move is made. 
    # Check for captured pieces:
    x = index % WIDTH
    y = int(index / WIDTH)
    captured = False
    if x > 0: # Check for capture 1 sq to the left
        capture = Flood(board, (x-1)+y*WIDTH, ENEMY)
        if capture != -1:
            captured = True
            for i in capture[1]:
                board[i] = EMPTY
    if x < WIDTH-1: # Check for capture 1 sq to the right
        capture = Flood(board, (x+1)+y*WIDTH, ENEMY)
        if capture != -1:
            captured = True
            for i in capture[1]:
                board[i] = EMPTY
    if y > 0: # Check for capture 1 sq up
        capture = Flood(board, x+(y-1)*WIDTH, ENEMY)
        if capture != -1:
            captured = True
            for i in capture[1]:
                board[i] = EMPTY
    if y < WIDTH-1: # Check for capture 1 sq down
        capture = Flood(board, x+(y+1)*WIDTH, ENEMY)
        if capture != -1:
            captured = True
            for i in capture[1]:
                board[i] = EMPTY
    if not captured: #Check for suicides
        capture = Flood(board, index, color)
        if capture[0] != -1:
            return -1, bd
    return 1, board


def createMask(board, positions): # This function isn't working properly, I suspect because it checks at the wrong index.
    color = board[0]
    mask = np.zeros(WIDTH*WIDTH+1)
    mask[0] = 1
    for a in range(1, WIDTH*WIDTH):
        mask[a] = 0
        if board[a] == EMPTY:
            mask[a] = 1
        current = ""
        for b in range(WIDTH*WIDTH): # Check for suicides or ko moves
            if mask[a] == 1:
                variation = Move(board[:], b)
                if variation[0] == -1:
                    mask[a] = 0
                else:
                    if len(positions) > 0:
                        position = encode(variation[1])
                        if position in positions:
                            mask[a] = 0
    return mask

In [ ]:
def printBoard(board, turn):
    string = ""
    for y in range(-1, WIDTH):
        for x in range(-1, WIDTH):
            if x < 0:
                if y < 0:
                    string += '# '
                else:
                    string += str(y)+" "
            elif y < 0 and x > -1:
                string += ' ' + chr(65+x)
            else:
                if board[x+y*WIDTH] == BLACK:
                    string += ' @'
                elif board[x+y*WIDTH] == WHITE:
                    string += ' O'
                elif board[x+y*WIDTH] == EMPTY:
                    string += ' .'
        string += '\n'
    print(string)

def stringToIndex(mv): 
    # Output 0 for pass,
    # 1-81 for a1-i8
    #-1 if invalid
    move = mv.lower()
    xC = 0
    yC = 0
    index = 0
    
    if move == "pass":
        return 0
    else:
        xC = ord(move[0]) - ord('a')
        yC = ord(move[1]) - ord('0')
        index = 1 + xC+yC*WIDTH
    if index < 1 or index > 81:
        return -1
    return index

def main():
    board = np.zeros(WIDTH*WIDTH+1)
    positions = []
    board[-1] = -1
    
    val = ""
    xC = 0
    yC = 0
    
    mask = createMask(board, positions)
    net_moveList = (mask * model(np.array([board]), training=False)[0]) # Multiply model prediction by the mask.
    net_move = np.argmax(net_moveList)
    print(net_move)
    board = Move(board, net_move)[1]
    print(board)
    while val != "quit":
        positions.append(encode(board))
        mask = createMask(board, positions)
        printBoard(board, 0)
        string = "Enter "
        if board[-1] == BLACK: # The net plays black's moves here.
            string += "Black's Move"
            net_moveList = (mask * model(np.array([board]), training=False)[0])
            net_move = np.argmax(net_moveList)
            board = Move(board, net_move)[1]
            string += ": " + str(net_move) # Print what index the net choses to play at.
            print(string)
        else:
            string += "White's Move"
            val = input(string)
            if val == "pass":
                board[-1] *= -1
            elif val != "quit":
                val = stringToIndex(val)
                if val >= 1 and val <= 81 and mask[xC+yC*WIDTH] == 1: # Check if the move is legal
                    board = Move(board, val-1)[1] # Make the move
                else:
                    print("Illegal move!", end = "\n")
main()

32
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
#  A B C D E F G H I
0  . . . . . . . . .
1  . . . . . . . . .
2  . . . . . . . . .
3  . . . . . @ . . .
4  . . . . . . . . .
5  . . . . . . . . .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .



Enter White's Move f2


#  A B C D E F G H I
0  . . . . . . . . .
1  . . . . . . . . .
2  . . . . . O . . .
3  . . . . . @ . . .
4  . . . . . . . . .
5  . . . . . . . . .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .

Enter Black's Move: 34
#  A B C D E F G H I
0  . . . . . . . . .
1  . . . . . . . . .
2  . . . . . O . . .
3  . . . . . @ . @ .
4  . . . . . . . . .
5  . . . . . . . . .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .



Enter White's Move g3


#  A B C D E F G H I
0  . . . . . . . . .
1  . . . . . . . . .
2  . . . . . O . . .
3  . . . . . @ O @ .
4  . . . . . . . . .
5  . . . . . . . . .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .

Enter Black's Move: 24
#  A B C D E F G H I
0  . . . . . . . . .
1  . . . . . . . . .
2  . . . . . O @ . .
3  . . . . . @ O @ .
4  . . . . . . . . .
5  . . . . . . . . .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .



...

#  A B C D E F G H I
0  . . . . . . . . .
1  . . . . . . @ . .
2  . . . . . O @ . .
3  . . . . O @ O @ .
4  . . . . . O . . .
5  . . . . . . . . .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .

Enter Black's Move: 52
#  A B C D E F G H I
0  . . . . . . . . .
1  . . . . . . @ . .
2  . . . . . O @ . .
3  . . . . O @ O @ .
4  . . . . . O . . .
5  . . . . . . . @ .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .



Enter White's Move f3


#  A B C D E F G H I
0  . . . . . . . . .
1  . . . . . . @ . .
2  . . . . . O @ . .
3  . . . . O O O @ .
4  . . . . . O . . .
5  . . . . . . . @ .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .

Enter Black's Move: 0
#  A B C D E F G H I
0  @ . . . . . . . .
1  . . . . . . @ . .
2  . . . . . O @ . .
3  . . . . O O O @ .
4  . . . . . O . . .
5  . . . . . . . @ .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .



Enter White's Move g2


#  A B C D E F G H I
0  @ . . . . . . . .
1  . . . . . . @ . .
2  . . . . . O O . .
3  . . . . O O O @ .
4  . . . . . O . . .
5  . . . . . . . @ .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .

Enter Black's Move: 0
#  A B C D E F G H I
0  @ . . . . . . . .
1  . . . . . . @ . .
2  . . . . . O O . .
3  . . . . O O O @ .
4  . . . . . O . . .
5  . . . . . . . @ .
6  . . . . . . . . .
7  . . . . . . . . .
8  . . . . . . . . .

